In [ ]:
# Imports 
# IMPORTING STUFF

import os 
import csv
import math
import time
import copy
import glob 
import pickle
import shutil
import random
import ortools                       
import logging
import datetime
import matplotlib 
import numpy as np
import pandas as pd
import configparser  
import plotly.express as px  
from shapely import geometry
from functools import reduce
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim  

matplotlib.rc('xtick', labelsize=26) 
matplotlib.rc('ytick', labelsize=26) 

plt.rcParams['font.size'] = '26'
plt.rcParams['figure.figsize'] = (10,7.5)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50

In [ ]:
city = 'ahm'

flag = 0
w1, w2 = 0.5, 0.5
'''
"flag" decides which distance metric/measure to consider:
0: euclidean distance (or physical distance)
1: rating
2: combination of euclidean distance and rating
   where 'w1' is weight given to euclidean distance and 'w2' is weight given to rating
'''

In [ ]:
# LOADING DATASETS
import glob 

driver_files_ahm = sorted(glob.glob("data/driver_locs/ahm/driver_data_ahm_day*.csv"))
num_days = len(driver_files_ahm) 
driver_dfs_ahm = [pd.read_csv(driver_files_ahm[idx]) for idx in range(num_days)]
driver_dfs_dict = {'ahm': driver_dfs_ahm}
zone_df_ahm = pd.read_csv("data/zone_data/zone_data_ahm.csv")
zone_dfs_dict = {'ahm': zone_df_ahm}
income_df_ahm = pd.read_csv("data/income_data/incomes_ahm.csv")
income_dfs_dict = {'ahm': income_df_ahm} 
base_zone_ahm = pd.read_csv("data/base_zones/ahm_base_zones.csv")
base_zones_dict = {'ahm': base_zone_ahm}

In [ ]:
# UTILITIES
#'city' takes values in {'ahm', 'blr', 'del'}

def driver_union(drivers_dict):
    """
    Finding union of all the drivers over the days 
    """
    driver_dfs = drivers_dict[city] 
    num_days = len(driver_dfs)
    
    driver_udf = pd.concat([driver_dfs[idx] for idx in range(num_days)])
    driver_udf = driver_udf.drop('Unnamed: 0', axis=1)
    driver_udf = driver_udf.drop_duplicates('de_id').reset_index().drop('index', axis=1)
    
    return driver_udf


def driver_intersection(drivers_dict):
    """
    Finding intersection of all the drivers over the days
    """
    driver_dfs = drivers_dict[city]
    driver_idf = reduce(lambda left,right: pd.merge(left,right,on='de_id'), driver_dfs)
    driver_idf = driver_idf[['de_id', 'lat_x', 'lng_x']]
    driver_idf = driver_idf.loc[:, ~driver_idf.columns.duplicated()] 
    driver_idf = driver_idf.rename(columns={'lat_x':'lat', 'lng_x':'lng'})
    
    return driver_idf



def drivers_zones(drivers_dict, zones_dict):
    """
    To get the data to be input to fair_clustering: "driver_locs" and "zone_locs"
    """
    driver_idf = driver_intersection(drivers_dict) 
    
    # finding "driver_locs":
    driver_locs = driver_idf[['lat', 'lng']] 
    driver_locs = driver_locs.values 
    
    # finding "zone_locs":
    zone_df = zones_dict[city]
    zone_locs = np.array(zone_df[['lat', 'lng']])
    
    return driver_locs, zone_locs


def get_capacities(zones_dict):
    """
    returns "lower_caps" and "upper_caps"
    lower_caps: [1 x num_centres] array with lower capacity of each zone
    upper_caps: [1 x num_centres] array with upper capacity of each zone
    """
    zone_df = zones_dict[city] 

    lower_caps = 0.3*zone_df['avg_cap'].values
    upper_caps = 1.0*zone_df['avg_cap'].values
    
    return lower_caps, upper_caps

In [ ]:
# LOGGER

import logging
logging.basicConfig(filename=city+".log", format='%(asctime)s  %(message)s', filemode='w')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

**Getting the inputs to fair_clustering**

In [ ]:
driver_udf = driver_union(driver_dfs_dict)
driver_idf = driver_intersection(driver_dfs_dict) 

driver_locs, zone_locs = drivers_zones(driver_dfs_dict, zone_dfs_dict)
num_drivers, num_centres = driver_locs.shape[0], zone_locs.shape[0]

lower_caps, upper_caps = get_capacities(zone_dfs_dict)
assert driver_locs.shape[0]>lower_caps.sum() and driver_locs.shape[0]<upper_caps.sum(), \
"This set of num_drivers, lower_caps and upper_caps will lead to an infeasible solution !"

In [ ]:
def euclidean_distance(d_loc, z_loc):
    lat1, lng1 = d_loc[0], d_loc[1]
    lat2, lng2 = z_loc[0], z_loc[1]

    dist = np.sqrt(np.power(lat1-lat2, 2) + np.power(lng1-lng2, 2))
    return dist

def L2Distance(data):
  # "data": latitude-longitude level locations 
  transposed = np.expand_dims(data, axis = 1)
  distance = np.power(data - transposed, 2)
  distance = np.power(np.abs(distance).sum(axis = 2), 0.5) 
  return distance 

driver_dists = L2Distance(driver_locs)

In [ ]:
# assigning ratings to sellers:
from scipy.stats import truncnorm
from numpy.random import SeedSequence 
from numpy.random import default_rng

def get_truncated_normal(mean, sd, low, upp):
    return truncnorm( (low-mean)/sd, (upp-mean)/sd, loc=mean, scale=sd) 

def generate_ratings(num_drivers):
    mean = 3.5
    sd = 1
    min_rating = 0.0
    max_rating = 5.0
    seedVal = 36778738061272522495168595294022739449
    rng = default_rng(seedVal)
    dist = get_truncated_normal(mean, sd, min_rating, max_rating)
    ratings = dist.rvs(num_drivers, random_state=rng)
    ratings = [round(x, 1) for x in ratings]
    return ratings

def abs_difference(ratings):
    transposed = np.expand_dims(ratings, axis=1)
    diff = abs(ratings-transposed) 
    return diff   

def minmax(distance, fair_distance):
    num_samples = len(distance)
    mx, mn = distance.max(), distance.min()
    dists = distance.flatten()
    dists = np.asarray( [((x-mn)/(mx-mn)) for x in dists] )
    distance = dists.reshape((num_samples, num_samples))
    fair_distance = (fair_distance-mn)/(mx-mn)
    return distance, fair_distance

---

**Algorithm**

In [ ]:
# will go with the default parameters of cplex:
from cplex import Cplex
model = Cplex()
model.parameters.simplex.tolerances.feasibility.get(),\
model.parameters.simplex.tolerances.optimality.get(),\
model.parameters.simplex.tolerances.markowitz.get()      

model.parameters.workmem.set(10240) # 10GB  
model.parameters.emphasis.memory.set(1)

In [ ]:
# Fair Clustering - LPP contstraints and Cplex
from cplex import Cplex
# from lp_tools import *
from lp_tools_kn import * 

alpha_fair = 2

def fair_clustering(dataset, centres, lower_cap, upper_cap, fair_distance, prohibited_assignments):
  # Step 1: 	 Create an instance of Cplex 
  problem = Cplex()
  problem.parameters.simplex.tolerances.feasibility.set(float(1e-9))
  problem.parameters.simplex.tolerances.optimality.set(float(1e-9))
  problem.parameters.simplex.tolerances.markowitz.set(float(0.9999)) 
  problem.parameters.emphasis.memory.set(1)
  problem.parameters.workmem.set(10240)

  # Step 2: 	 Declare that this is a minimization problem
  problem.objective.set_sense(problem.objective.sense.minimize)
    
  """
   Step 3.   Declare and  add variables to the model. 
        The function prepare_to_add_variables (dataset, centres) prepares all the required information for this stage.
  
    objective: a list of coefficients (float) in the linear objective function
    lower bound: a list of floats containing the lower bounds for each variable
    upper bound: a list of floats containing the upper bounds for each variable
    variable_names: a list of strings that contains the name of the variables
  """
  ## if working with "lp_tools":
  print("Adding Variables...")
  
  # objective, lower_bound, upper_bound, variable_names, P,C = prepare_to_add_variables(dataset, centres)
  ## if working with "lp_tools_kn": 
  objective, lower_bound, upper_bound, variable_names, P,C = prepare_to_add_variables(dataset, centres, prohibited_assignments)
  problem.variables.add(
      obj = objective,
      lb = lower_bound,
      ub = upper_bound,
      names = variable_names
     
    )
  
  print("Variables Added !")
    
    
  """
  Step 4.   Declare and add constraints to the model.
            There are few ways of adding constraints: row wise, col wise and non-zero entry wise.
            Assume the constraint matrix is A. We add the constraints non-zero entry wise.
            The function prepare_to_add_constraints(dataset, centres) prepares the required data for this step.
  
   coefficients: Three tuple containing the row number, column number and the value of the constraint matrix
   senses: a list of strings that identifies whether the corresponding constraint is
           an equality or inequality. "E" : equals to (=), "L" : less than (<=), "G" : greater than equals (>=)
   rhs: a list of floats corresponding to the rhs of the constraints.
   constraint_names: a list of string corresponding to the name of the constraint
  """
  print("Adding Constraints...")
    
  rhs, senses, row_names, coefficients = prepare_to_add_constraints(dataset, centres, upper_cap,lower_cap, P,C, alpha_fair, fair_distance, ratings, flag)
  print("num_constraints:", len(senses)) 
  logger.info(f"\t\t\tnum_constraints = {len(senses)}")
  problem.linear_constraints.add(
      rhs = rhs,
      senses = senses,
      names = row_names
    )
  problem.linear_constraints.set_coefficients(coefficients)

  print("Constraints Added !")
    
  # Step 5.	Solve the problem
  problem.solve()

  result = {
    "status": problem.solution.get_status(),
    "success": problem.solution.get_status_string(),
    "objective": problem.solution.get_objective_value(),
    "assignment": problem.solution.get_values(),
  }
    
  qm = problem.solution.quality_metric  
  print("Solution Quality:", problem.solution.get_float_quality([qm.max_x, qm.max_primal_infeasibility]))
  
  # print("Status:", result['status']) # outputs a number: "1" for optimal solution, "2" for unbounded ray and "3" for infeasible solution
  solution_status = result['status']
  assert solution_status==1, "Solution isn't optimal !"

  print("Status:", problem.solution.get_status_string()) # optimal, unbounded ray, infeasible

  return result
'''

In [ ]:
# Fair Assignment of drivers to the FFCs / warehouses
import copy
import dependent_routing as dp

# configParser.read(configFilePath)

num_samples, num_centres = driver_locs.shape[0], zone_locs.shape[0]

def fair_assignment(prob_dis,driver_loc):
  '''Assigning the driver using the probaility distribution using dependent rounding'''  
  
  # "prob_dis" is the result of the Fair-LP program "fair_clustering"  
  prob_dist = copy.deepcopy(prob_dis)
  # print("prob_dist shape [num_drivers x num_ffc]:", prob_dist.shape)

  rounding = dp.DependentRounding(prob_dist)
  rounding._buildGraph(prob_dist)
  final_assignment = rounding.round()
  final_assignment = np.around(final_assignment,2)

  driver_df = pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df['ffc_index'] = -1 # unassigned

  for i in range(num_samples):
    for j in range(num_centres):
      # choose values which are close to 1
      if abs(final_assignment[i][j]-1) < 0.01: 
        driver_df.at[i,'ffc_index'] = j
        
  return driver_df, final_assignment


In [ ]:
def sanityCheck(probs):
    """
    To cope with bound violations which can occur upto the feasibility parameter range 
    So the lower bound of 0.0 on the probabilities can get violated and the values can go down to (0-feasibility_parameter_value)
    """
    for i in range(len(probs)):
        last_pos_index = -1
        neg_value = 0
        
        for j in range(len(probs[0])):
            assert probs[i][j] >= -1e-6 
            
            if probs[i][j] < 0:
                neg_value += probs[i][j]
                probs[i][j] = 0
            elif probs[i][j] > 0:
                last_pos_index = j

        max_pos_index = np.argmax(probs[i])
        probs[i][max_pos_index] += neg_value
        
        assert probs[i][max_pos_index] > 0
        
    return probs

In [ ]:
# main :
def FairAssign_solver(driver_locs, zone_locs, lower_cap, upper_cap, fair_distance, prohibited_assignments):
    # Fair-LP:
    try:
        lp_output = fair_clustering(driver_locs, zone_locs, lower_cap, upper_cap, fair_distance, prohibited_assignments)
    except:
        logger.error("Solution Non-optimal (Unbounded Ray or Infeasible) !")
        return None, None
        
    prob_dis = np.reshape(lp_output['assignment'][:num_samples*num_centres], (-1, num_centres))
    
    try:
        prob_dist = sanityCheck(copy.deepcopy(prob_dis)) # this might raise an assertion error
    except:
        logger.error("Sanity Check Assertion !")
        return None, None
    
    # Randomized Dependent Rounding:
    try:
        df = fair_assignment(prob_dist, driver_locs)[0] # this might raise an assertion error
        final_assignment = df['ffc_index'].values
    except:
        logger.error("Dependent Rounding Assertion !")
        return prob_dist, None
    
    return prob_dist, final_assignment
    

In [ ]:
k_list = [(x-1) for x in [7, 5, 3]] # x-1 NOT x bcz 'k' is a 0-based index
fd_list = [(driver_dists.mean()/alpha) for alpha in [28, 14, 8]] 
num_runs = 5

k_fd_dict = {k:\
                {fd_idx:\
                    {num_run:\
                        {'p_dist':None, 'assignment':None}
                        for num_run in range(num_runs)
                    } 
                    for fd_idx in range(len(fd_list))
                } 
            for k in k_list
            }

for k in k_list:
    print(k)
    logger.info(f"Considering k = {k+1} nearest zones")
    prhbtd_assigns = get_pa(dz_dist, k)
    
    for f_idx, fair_distance in enumerate(fd_list):
        logger.info(f"\tfair_distance = {fair_distance}")
        
        for num_run in range(num_runs):
            logger.info(f"\t\tnum_run = {num_run}")
            
            prob_dist, final_assignment = FairAssign_solver(driver_locs, zone_locs, lower_caps, upper_caps, fair_distance, prhbtd_assigns)
            
            k_fd_dict[k][f_idx][num_run]['p_dist'] = prob_dist
            k_fd_dict[k][f_idx][num_run]['assignment'] = final_assignment
    
    # Store intermediate results as well as fail-safe:
    # saving current state of "k_fd_dict":
    pickling_on = open(f"dict_k={k+1}.pickle", "wb")
    pickle.dump(k_fd_dict, pickling_on)
    pickling_on.close() 

pickling_on = open("Assignments_ratings_"+city+".pickle", "wb")
pickle.dump(k_fd_dict, pickling_on)
pickling_on.close()

---

**Evaluation**

In [ ]:
# METRICS:

def gini_index(incomes):
    num = len(incomes)
    total = incomes.sum() 
    inc_sum = 0.0
    for i in range(num):
        for j in range(num):
            inc_sum += abs(incomes[i]-incomes[j])
    gini = inc_sum / (2*num*total)
    return gini


def avg_distance(zone_labels, driver_locs, zone_locs):
    """
    returns the 'cost' of the assignment
    zone_labels: indices of the assigned zones
    a zone_label 'z' has location zone_locs[z]
    """
    driver_dists = L2Distance(driver_locs) 
    num = len(zone_labels)
    dist = 0.0 
    for i in range(num):
        assigned_zone = zone_labels[i]
        driver_loc, zone_loc = driver_locs[i], zone_locs[int(assigned_zone)]
        driver_zone_dist = euclidean_distance(driver_loc, zone_loc)
        dist += np.sqrt(driver_zone_dist)
    avg_dist = dist/num
    return avg_dist                       
    

def spatial_inequality_index(incomes, driver_locs, ratings, combined, fair_distance):
    if flag==0:
        driver_dists = L2Distance(driver_locs)
    if flag==1:
        driver_dists = abs_difference(ratings)
    if flag==2:
        driver_dists = combined

    num = len(incomes)
    total = incomes.sum()

    term_i = 0.0    
    for i in range(num):
        sum_j = 0.0
        num_j = 1e-9    
        for j in range(i+1, num):
            if driver_dists[i][j] <= fair_distance and driver_dists[i][j]>0:
                num_j += 1
                sum_j += abs(incomes[i]-incomes[j])   
        term_i += (sum_j / num_j) 
    
    spin_idx = term_i / total 
    # spin_idx = round(spin_idx, 2)
    return spin_idx 


def income_gap(incomes, driver_locs, ratings, combined, fair_distance):
    """ 
    difference between incomes between any two drivers per unit distance (within fair_distance) 
    """
    alpha = 100

    if flag==0:
        driver_dists = L2Distance(driver_locs)
    if flag==1:
        driver_dists = abs_difference(ratings)
    if flag==2:
        driver_dists = combined

    driver_dists = driver_dists * alpha
    num = len(incomes)
    total = incomes.sum()

    terms = 0.0
    num_pair_drivers = 1e-7 # NOT 0 => to avoid division by 0
    for i in range(num-1):
        for j in range(i+1, num):
            if driver_dists[i][j]>0:
                num_pair_drivers += 1
                terms += (abs(incomes[i]-incomes[j])/driver_dists[i][j])
    
    inc_gap = terms/num_pair_drivers
    # inc_gap = round(inc_gap, 2)
    return inc_gap



In [ ]:
# Random generation of locations within a given zone

# generating random locations withing a zone (given the zone boundary):
def random_loc_generator(zone_bdry):
  lats, longs = path_related_preprocessing(zone_bdry)
  coords = [(x,y) for x,y in zip(lats, longs)]
  min_lat, max_lat = min(lats), max(lats)
  min_lng, max_lng = min(longs), max(longs)
  new_lat, new_lng = random.uniform(min_lat, max_lat), random.uniform(min_lng, max_lng) 
  return [new_lat, new_lng]

# Checking if a given location lies inside a given zone:
def path_related_preprocessing(path_bdry):
  # exemplar path_bdry: '12.954619258010608,77.6149292592163 12.954680993923494,77.61640664016727 ....'
  path_bdry = str(path_bdry)
  df = pd.DataFrame({'lts':[], 'lngs':[]})
  bdry_locs = path_bdry.split()
  lats, longs = [], []
  for loc in bdry_locs:
    lat, lng = loc.split(',')
    lats.append(float(lat))
    longs.append(float(lng))
  return lats, longs

def loc_in_zone(loc, zone_bdry):
  lats, longs = path_related_preprocessing(zone_bdry)
  coords = [(x,y) for x,y in zip(lats, longs)]
  polygon = geometry.MultiPoint(coords).convex_hull
  Point_X, Point_Y = loc[0], loc[1]
  point = geometry.Point(Point_X, Point_Y)
  return point.within(polygon)

# code to generate 'm' locations that lie within a given zone:
def generate_locs(m, zone_bdry):
    new_locs = []
    num_generated = 0
    while num_generated < m:
        new_loc = random_loc_generator(zone_bdry)
        sanity_check = loc_in_zone(new_loc, zone_bdry)
        if sanity_check:
            num_generated += 1
            new_locs.append(new_loc)
    return new_locs

**Data Preprocessing required for FoodMatch**